In [1]:
import pandas as pd
import numpy as np
import humanize
import plotly.express as px
import plotly.graph_objects as go

# Data For Good - Tax Observatory Dataviz

Visualizations for the EU Tax Observatory based on the [EU Tax Obs - Dataviz - Viz](https://docs.google.com/document/d/1q_7x8NrEWNsak9LSE3TxIiSphhVcqWqn6YeG1kQNQTk/edit#heading=h.97g77izer615) Google Docs.

## Table of contents
* [Initialization](#initialization)
* [Utils](#utils)
* [Vizualisations](#visualizations)
    * [#12 - available reports by company](#12---available-reports-by-company)
    * [#13 - company key financials kpis](#13---company-key-financials-kpis)
    * [#14 - company top jurisdictions for revenue](#14---company-top-jurisdictions-for-revenue)
    * [#15 - company’s % pre-tax profit and % employees by jurisdiction](#15---companys--pre-tax-profit-and--employees-by-jurisdiction)
    * [#18 - breakdown of revenue between related party and unrelated party in TH vs domestic vs non TH](#18---breakdown-of-revenue-between-related-party-and-unrelated-party-in-th-vs-domestic-vs-non-th)
    * [#21 - evolution of tax havens use over time : % profit vs % employees in TH over time](#21---evolution-of-tax-havens-use-over-time---profit-vs--employees-in-th-over-time)
    * [#22 - locations of profits booked vs. mean 3Y ETR](#22---locations-of-profits-booked-vs-mean-3y-etr)
* [Transparency score](#transparency-score)

## Initialization

In [2]:
# Import dataset in a DataFrame
df = pd.read_csv('./datasets/data_final_dataviz.csv', sep=',')

In the website it will be possible to select a specific company. To work on the viz creation, we simulate the selection by picking a random company but also a random year.

In [3]:
# Select a random company and a random year
def pick_random_company_and_year() -> tuple:
    company = np.random.choice(df['mnc'].unique())
    print(f'Selected company : {company}.')

    year = np.random.choice(df.loc[df['mnc'] == company, 'year'].unique())
    print(f'Selected year : {year}.')

    return company, year

In [4]:
# company, year = pick_random_company_and_year()
company, year = 'AXA', 2018

## Utils

In [5]:
def compute_missing_values(df: pd.DataFrame=df):
    df = df.loc[df['year'] == 2022, ['mnc', 'year', 'profit_before_tax', 'employees']]

    df['both'] = np.where(
        df['profit_before_tax'].isna() & df['employees'].isna(),
        np.nan,
        9999
    )

    mnc_list = df['mnc'].unique()

    data = pd.DataFrame()

    for mnc in mnc_list:
        n_rows = len(df.loc[df['mnc'] == mnc])
        data[mnc] = df.loc[
            df['mnc'] == mnc, ['profit_before_tax', 'employees', 'both']
        ].isna().sum() / n_rows

    return data.T

In [6]:
# df_na = compute_missing_values()
# df_na.to_csv('./datasets/na2022.csv')

## Visualizations

### #12 - available reports by company

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|tbd (goal is on company page, to show reports tracked, so could be just a table showing year available for a company)|-||`company explo` need a filter by company||

In [7]:
def compute_company_available_reports(df: pd.DataFrame, company: str) -> dict:
    """Compute the number of reports tracked for a specific company and the 
    available fiscal years.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): company name.

    Returns:
        dict: numbers of reports and fiscal years.
    """
    available_years = df.loc[df['mnc'] == company, 'year'].unique()
    n_reports = len(available_years)

    # Convert type of items from 'int' to 'str' in available years list
    years_string_list = [str(year) for year in available_years]
    
    # Summarize all available years in one string
    if len(years_string_list) == 1:
        years_string = years_string_list[0]
    elif len(years_string_list) > 1:
        years_string = ', '.join(years_string_list[:-1])
        years_string += ' and ' + years_string_list[-1]

    # Create a dictionnary with the results
    data = {
        'Company': company,
        'Reports': n_reports,
        'Fiscal year(s) available': years_string
    }

    return data


def display_company_available_reports(
        df: pd.DataFrame, company: str, hide_company: bool=True) -> pd.DataFrame:
    """Display the number of reports tracked for a specific company and the 
    available fiscal years.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): company name.
        hide_company (bool, optional): hide company name in final table. Defaults to True.

    Returns:
        pd.DataFrame: numbers of reports and fiscal years.
    """
    
    # Compute data
    data = compute_company_available_reports(df=df, company=company)

    # Create the table
    df = pd.DataFrame.from_dict(data=data, orient='index')

    if hide_company:
        return df[1:].style.hide(axis='columns')

    return df.style.hide(axis='columns')

In [8]:
display_company_available_reports(df=df, company=company, hide_company=True)

Reports,4
Fiscal year(s) available,"2018, 2019, 2020 and 2021"


### #13 - company key financials kpis

**kpis** : hq country, total revenue, total unrelated party revenues, total related party revenues, pre-tax profits, taxes paid, employees.

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|raw figure for each, or table with indicator name in col A, value in col B|hq country : -<br><br>metrics : sum (metric) by year by mnc|**giulia : keep all values, even negative values**|`company explo` need a filter by company||

In [9]:
def compute_company_key_financials_kpis(
        df: pd.DataFrame, company: str, year: int=None) -> dict:
    """Compute key financial KPIs for a company.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int, optional): fiscal year to filter the results with. Defaults to None.

    Returns:
        dict: company key financial KPIs.
    """
    
    kpis_list = ['total_revenues', 'unrelated_revenues', 'related_revenues', 
                 'profit_before_tax', 'tax_paid', 'employees']
    
    years_list = df.loc[df['mnc'] == company, 'year'].unique()

    # Compute sum of kpis
    if not year or year not in years_list:
        df = (df.loc[df['mnc'] == company]
            .groupby(['year', 'upe_name'], as_index=False)[kpis_list]
            .sum()
        )
    else:
        df = (df.loc[(df['mnc'] == company) & (df['year'] == year)]
              .groupby(['year', 'upe_name'], as_index=False)[kpis_list]
              .sum())

    df = df.set_index('year')

    # Make financial numbers easily readable with 'humanize' package
    for column in df.columns:
        if column not in ['employees', 'upe_name']:
            df[column] = df[column].apply(
                lambda x: humanize.intword(x) if isinstance(x, (int, float)) else x)
            df[column] = '€ ' + df[column]
        elif column == 'employees':
            df[column] = df[column].astype(int)

    # Clean columns string
    df = df.rename(columns={'upe_name': 'headquarter'})
    df.columns = df.columns.str.replace('_', ' ').str.capitalize()

    # Create a dictionnary with the results
    data = df.to_dict(orient='index')

    return data


def display_company_key_financials_kpis(
        df: pd.DataFrame, company: str, year: int=None) -> pd.DataFrame:
    """Display key financial KPIs for a company.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int, optional): fiscal year to filter the results with. Defaults to None.

    Returns:
        pd.DataFrame: company key financial KPIs.
    """
    
    # Compute data
    data = compute_company_key_financials_kpis(df=df, company=company, year=year)

    # Create the table
    df = pd.DataFrame.from_dict(data)

    return df

In [10]:
display_company_key_financials_kpis(df=df, company=company)

,2018,2019,2020,2021
Headquarter,France,France,France,France
Total revenues,€ 0,€ 0,€ 0,€ 112.1 billion
Unrelated revenues,€ 0,€ 0,€ 0,€ 99.9 billion
Related revenues,€ 0,€ 0,€ 0,€ 12.2 billion
Profit before tax,€ 1.2 billion,€ 5.1 billion,€ 4.5 billion,€ 9.1 billion
Tax paid,€ 744.0 million,€ 1.2 billion,€ 1.4 billion,€ 1.6 billion
Employees,0,0,0,92399


### #14 - company top jurisdictions for revenue

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|horizontal bar chart showing % total revenue for top 9 jurisdictions + rest as “OTHERS”|sum(revenue) by company x hq country x year, display top 9 and group remaining countries||`company explo` need a filter by company||

In [11]:
def compute_top_jurisdictions_revenue(
    df: pd.DataFrame, company: str, year: int) -> dict:
    """Rank jurisdictions on their percentage of total revenues in a top 10.
    When there are more than 10 jurisdictions, the tenth represent all 
    jurisdictions below 9.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year.

    Returns:
        dict: Top 10 jurisdictions for percentage of total revenues.
    """
    
    df = df.loc[
        (df['mnc'] == company) & (df['year'] == year), 
        ['jur_name', 'related_revenues', 'unrelated_revenues', 'total_revenues']
    ]

    # Calculate missing values in 'total_revenues' if 'related_revenues' and 
    # 'unrelated_revenues' are available
    df.loc[
        df['related_revenues'].notna() 
        & df['unrelated_revenues'].notna()
        & df['total_revenues'].isna(),
        'total_revenues'
    ] = df['related_revenues'] + df['unrelated_revenues']

    # Subset DataFrame
    df = df[['jur_name', 'total_revenues']]

    # Remove rows where 'total_revenues' is missing
    df = df.dropna(subset=['total_revenues'])

    # Group same 'jur_name' (sometimes several 'Other')
    # e.g. SWISS LIFE, 2021
    df = df.groupby('jur_name', as_index=False).sum()

    # Filter the top 10 'jur_name' for 'total_revenues'
    if len(df) > 10:
        # Check if 'Other' already in 'jur_name' and add the revenues
        # of the 'jur_name' below top 10 to its value
        if 'Other' in df['jur_name'].values:
            top = df.nlargest(10, 'total_revenues')
            below_top_revenues = df.loc[
                ~df['jur_name'].isin(top['jur_name']), 'total_revenues'].sum()
            top.loc[top['jur_name'] == 'Other', 'total_revenues'] += below_top_revenues
            top = top.reset_index(drop=True)
        else:
            # Keep top 9 and group all revenues of the rest in 'Others'
            top = df.nlargest(9, 'total_revenues')
            below_top_revenues = df.loc[
                ~df['jur_name'].isin(top['jur_name']), 'total_revenues'].sum()
            top = top.reset_index(drop=True)
            top.loc[9] = ['Others', below_top_revenues]
    else:
        top = df
    
    # Rename 'Other' to 'Others'
    top.loc[top['jur_name'] == 'Other', 'jur_name'] = 'Others'

    # Compute percentage of revenue
    top['total_revenues_%'] = top['total_revenues'] / top['total_revenues'].sum()
  
    # Convert DataFrame to dictionnary
    data = top.to_dict()
    
    return data


def display_jurisdictions_top_revenue(df: pd.DataFrame, company: str, year: int):
    """Display top 10 jurisdictions for percentage of total revenues in an
    horizontal bar chart.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year.
    """
    
    # Compute data
    data = compute_top_jurisdictions_revenue(df=df, company=company, year=year)

    # Create DataFrame
    df = pd.DataFrame.from_dict(data)
    df = df.sort_values(by='total_revenues_%')
    
    # Create figure
    fig = px.bar(df, x='total_revenues_%', y='jur_name',
                  orientation='h', title='Top jurisdictions for revenue', text_auto='.1%')

    # Update layout settings
    fig.update_layout(
        xaxis=dict(
            title='Percentage of total revenue',
            tickformat='.0%'
        ),
        yaxis_title=None,
        plot_bgcolor='white',
        width=800,
        height=480
    )

    # Define position of text values
    values_positions = [
        'outside' if value <= 0.05 else 'inside' for value in df['total_revenues_%']]

    fig.update_traces(
        textangle=0,
        textposition=values_positions,
        selector=dict(name='')
    )

    # Define style of hover on bars
    fig.update_traces(hovertemplate='%{y}: %{x: .3%}')

    fig.show()

In [12]:
display_jurisdictions_top_revenue(df=df, company=company, year=year)

### #15 - company’s % pre-tax profit and % employees by jurisdiction

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|horizontal bar chart with one bar for % pre tax profit, the other for % employees, in different colors rank by % pre-tax profit descending|for a company : filter (profit >0) profits by JUR / total profit vs employees by JUR / total employees|**giulia : keep only positive values**|`company explo` need a filter by company||

In [13]:
def compute_pretax_profit_and_employees_rank(
        df: pd.DataFrame, company: str, year: int) -> dict:
    """Compute jurisdictions percentage of profit before tax and percentage
    of employees and rank by percentage of profit.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year.

    Returns:
        dict: rank of jurisdictions with percentage of profit before and percentage
        of employees.
    """
    
    # Filter rows with selected company/year and subset with necessary features
    features = ['jur_name', 'profit_before_tax', 'employees']
    df = df.loc[(df['mnc'] == company) & (df['year'] == year), features]
    
    # Keep only profitable jurisdictions
    df = df.loc[df['profit_before_tax'] >= 0]

    # Sort jurisdictions by profits
    df = df.sort_values(by='profit_before_tax').reset_index(drop=True)

    # Calculate percentages
    df['profit_before_tax_%'] = df['profit_before_tax'] / df['profit_before_tax'].sum()
    df['employees_%'] = df['employees'] / df['employees'].sum()
    df = df.drop(columns=['profit_before_tax', 'employees'])

    data = df.to_dict()

    return df


def display_pretax_profit_and_employees_rank(
        df: pd.DataFrame, company: str, year: int):
    """Display rank of jurisdictions by percentage of profit before and percentage
        of employees.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year.
    """
    
    # Compute data
    data = compute_pretax_profit_and_employees_rank(df=df, company=company, year=year)

    # Create DataFrame
    df = pd.DataFrame(data)

    # Rename columns
    df = df.rename(columns={
        'profit_before_tax_%': 'Percentage of pre-tax profit', 
        'employees_%': 'Percentage of employees'
    })

    # Create figure
    fig = px.bar(
        df,
        x=['Percentage of employees', 'Percentage of pre-tax profit'],
        y='jur_name',
        barmode='group',
        orientation='h',
        text_auto='.1%'
    )
    
    # Set figure height (min. 480) depending on the number of jurisdictions
    fig_height = max(480, (48*len(df['jur_name'])))

    # Set maximum value for x axis
    if not df[['Percentage of pre-tax profit', 'Percentage of employees']].isna().all().all():
        max_x_value = max(df[['Percentage of pre-tax profit', 'Percentage of employees']].max(axis='columns')) + 0.1
    else:
        max_x_value = 1
    
    # Update layout settings
    fig.update_layout(
        title='Profitables jurisdictions pre-tax profit & employees',
        xaxis=dict(
            title=None,
            tickformat='.0%',
            range=[0, max_x_value ]
        ),
        yaxis_title=None,
        legend=dict(
            title=dict(text=''),
            orientation='h'
        ),
        plot_bgcolor='white',
        width=800,
        height=fig_height
    )

    # Add annotations for NaN values where there should have been a bar
    for index, row in df.iterrows():
        if pd.isna(row['Percentage of employees']):
            fig.add_annotation(
                xanchor='left',
                x=0.001,
                y=df.index[index],
                yshift=-10,
                text='Information not provided',
                showarrow=False,
                font=dict(size=12)
            )
        if pd.isna(row['Percentage of pre-tax profit']):
            fig.add_annotation(
                xanchor='left',
                x=0.001,
                y=df.index[index],
                yshift=10,
                text='Information not provided',
                showarrow=False,
                font=dict(size=12)
            )

    # Loop through each bar trace and hide the text if the value is NaN
    for trace in fig.data:
        values = df[trace.name]
        text_position = ['outside' if not np.isnan(value) else 'none' for value in values]
        trace.textposition = text_position

    fig.show()

In [14]:
display_pretax_profit_and_employees_rank(df=df, company=company, year=year)

### #18 - breakdown of revenue between related party and unrelated party in TH vs domestic vs non TH

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|horizontal barchart % of revenue from related-party revenue, and % revenue from unrelated party, show value for domestic (hq country) vs tax havens vs. non havens|get total revenue in TH, in domestic country, in non TH then show the % coming from unrelated party revenue and related party revenue (sum of both should equal total revenue)||`company explo` need a filter by company|what if sum or unrelated party revenue  and related party revenue does not equal to total revenue ? safeguard ?|

In [15]:
def compute_related_and_unrelated_revenues_breakdown(
        df: pd.DataFrame, company: str, year: int) -> dict:
    """Compute related and unrelated revenues in tax heaven, non tax heaven and
    domestic jusrisdictions.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year to filter the results with.

    Returns:
        dict: revenues percentage for different type of jurisdictions.
    """

    # Filter rows with selected company/year and subset with necessary features
    features = ['upe_code', 'jur_code', 'jur_name', 'jur_tax_haven', 
                'unrelated_revenues', 'related_revenues']
    
    df = df.loc[(df['mnc'] == company) & (df['year'] == year), features]

    # Drop rows where either unrelated or related revenues are missing
    df = df.dropna(subset=['unrelated_revenues', 'related_revenues'])

    # 'total_revenues' is recreated using related and unrelated revenues since the one
    # reported by companies is not always reliable
    df['total_revenues'] = df['unrelated_revenues'] + df['related_revenues']
    
    # Create a column to check if 'jur_code' is the domestic country
    df['domestic'] = df.apply(lambda row: row['jur_code'] == row['upe_code'], axis='columns')

    # Compute kpis in a new DataFrame
    data = pd.DataFrame()
    data['tax_haven'] = df.loc[df['jur_tax_haven'] == True, ['unrelated_revenues', 'related_revenues']].sum()
    data['non_tax_haven'] = df.loc[df['jur_tax_haven'] == False, ['unrelated_revenues', 'related_revenues']].sum()
    data['domestic'] = df.loc[df['domestic'] == True, ['unrelated_revenues', 'related_revenues']].sum()

    # Replace values with share (%) of 'unrelated/related revenues'
    data = data.div(data.sum(axis='rows'), axis='columns')

    # Rename indexes
    data = data.rename(index={
        'unrelated_revenues': 'unrelated_revenues_percentage',
        'related_revenues': 'related_revenues_percentage'
    })

    # Convert DataFrame to dictionnary
    data = data.to_dict()

    return data


def display_related_and_unrelated_revenues_breakdown(df: pd.DataFrame, company: str, year: int):
    """Display related and unrelated revenues in tax heaven, non tax heaven and
    domestic jusrisdictions.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name
        year (int): fiscal year to filter the results with.
    """
    
    # Compute data
    data = compute_related_and_unrelated_revenues_breakdown(df=df, company=company, year=year)

    # Create DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')

    # Rename columns and indexes
    df.columns = df.columns.str.replace('_', ' ').str.capitalize()
    df.index = df.index.str.replace('_', ' ').str.capitalize()
    
    # Create figure
    fig = px.bar(
        df,
        x=['Unrelated revenues percentage', 'Related revenues percentage'],
        y=df.index,
        orientation='h',
        text_auto='.0%'
    )

    # Update layout settings
    fig.update_layout(
        title='Breakdown of revenue',
        xaxis=dict(
            title=None,
            tickformat='.0%'
        ),
        yaxis_title=None,
        legend=dict(
            title=dict(text=''),
            orientation='h'
        ),
        plot_bgcolor='white',
        width=800,
        height=480
    )

    # Define position of text values
    for col in ['Unrelated revenues percentage', 'Related revenues percentage']:
        
        values_positions = ['outside' if value <= 0.05 else 'inside' for value in df[col]]
    
        fig.update_traces(
            textangle=0,
            textposition=values_positions,
            selector=dict(name=col)
        )

    # Add annotation if no values are availables (no bar displayed)
    for i, index in enumerate(df.index):
        if df.loc[index].isna().all():
            fig.add_annotation(
                    x=0.5,
                    y=df.index[i],
                    text='No information to display',
                    showarrow=False,
                    font=dict(size=13)
                )
    
    fig.show()

In [16]:
display_related_and_unrelated_revenues_breakdown(df=df, company=company, year=year)

### #21 - evolution of tax havens use over time : % profit vs % employees in TH over time

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|line chart ?|||`company explo` need a filter by company||

In [17]:
def compute_tax_havens_use_evolution(df: pd.DataFrame, company: str) -> dict:
    """Compute the evolution of tax havens use by company over time.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name.

    Returns:
        dict: tax havens percentage of profits and employees for each year.
    """
    
    # Filter rows with selected company and subset with necessary features
    features = ['jur_code', 'year', 'jur_tax_haven', 'profit_before_tax', 'employees']
    df = df.loc[(df['mnc'] == company), features]
    
    # Keep jurisdictions with profitable or missing profits
    df = df.loc[(df['profit_before_tax'] >= 0) | (df['profit_before_tax'].isna())]

    # For all sum calculations below :
    # - Result NA : all jurisdictions values were NA ;
    # - Result 0 : at least one jurisdiction was reported as 0.

    # Calculate total profit and employees by year and tax haven status
    df = df.groupby(['year', 'jur_tax_haven'], as_index=False)[['profit_before_tax', 'employees']].sum(min_count=1)
    
    # Calculate total profits and employees for each year
    for year in df['year'].unique():
        df.loc[df['year'] == year, 'total_profit'] = df.loc[df['year'] == year, 'profit_before_tax'].sum(min_count=1)
        df.loc[df['year'] == year, 'total_employees'] = df.loc[df['year'] == year, 'employees'].sum(min_count=1)

    # Remove non tax haven jurisdictions
    df = df.loc[df['jur_tax_haven'] == True].reset_index()
    
    # Calculate percentages
    df['tax_havens_profit_%'] = df['profit_before_tax'] / df['total_profit']
    df['tax_havens_employees_%'] = df['employees'] / df['total_employees'] 

    # Convert necessary data to dictionnary
    data = df[['year', 'tax_havens_profit_%', 'tax_havens_employees_%']].to_dict()

    return data


def display_tax_havens_use_evolution(df: pd.DataFrame, company: str):
    """Display the evolution of tax havens use by company over time.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name.
    """
    
    # Compute data
    data = compute_tax_havens_use_evolution(df=df, company=company)

    # Create DataFrame
    df = pd.DataFrame.from_dict(data)

    # Rename columns
    df = df.rename(columns={
        'tax_havens_profit_%': 'Percentage of profits in tax havens', 
        'tax_havens_employees_%': 'Percentage of employees in tax havens'
    })
    
    # Create figure
    fig = px.bar(
        df,
        x='year',
        y=['Percentage of profits in tax havens', 'Percentage of employees in tax havens'], 
        barmode='group', 
        text_auto='.1%'
    )
    
    # Update layout settings
    fig.update_layout(
        title='Tax havens use in profitables jurisdictions',
        xaxis_title=None,
        yaxis_title=None,
        yaxis_tickformat ='.0%',
        legend=dict(
            title=dict(text=''),
            orientation='h'
        ),
        plot_bgcolor='white',
        width=800,
        height=480
    )
    
    fig.show()

In [18]:
print(company)
display_tax_havens_use_evolution(df=df, company=company)

AXA


### #22 - locations of profits booked vs. mean 3Y ETR

|viz|how to compute|how to manage negative values|where / variant needed|comment|
|-|-|-|-|-|
|plot : x-axis : mean % profit, y-axis : mean ETR over 3Y<br>1 bubble per jurisdiction, with size of the bubble based on profit per employee in the JUR and a color code for tax havens vs others|by company, by JUR, calculate % total profit over several years, and tax rate (paid taxes/ pre-tax profit)||`company explo` need a filter by company||

In [19]:
def compute_profits_locations(df:pd.DataFrame, company:str) -> dict:
    
    # Filter rows with selected company and subset with necessary features
    features = ['jur_code', 'jur_name', 'year', 'jur_tax_haven', 'profit_before_tax', 
                'tax_paid', 'employees']
    df = df.loc[(df['mnc'] == company), features]

    # Keep jurisdictions with profitable or missing profits
    df = df.loc[(df['profit_before_tax'] >= 0) | (df['profit_before_tax'].isna())]

    # Remove jurisdictions with missing values for profits or employees
    df = df.loc[(df['profit_before_tax'].notna()) & (df['employees'].notna())]

    # Replace 0 employees by 1
    df.loc[df['employees'] == 0, 'employees'] = 1

    # Calculate profit per employee
    df['profit_per_employee'] = df['profit_before_tax'] / df['employees']

    # Calculate tax rate
    df['tax_rate'] = df['tax_paid'] / df['profit_before_tax']

    return df

In [20]:
print(company)
compute_profits_locations(df=df, company=company)

AXA


,jur_code,jur_name,year,jur_tax_haven,profit_before_tax,tax_paid,employees,profit_per_employee,tax_rate
1232,FRA,France,2021,False,2.046991e+09,307064000.0,19969.0,1.025084e+05,0.150007
1235,CZE,Czechia,2021,False,4.003000e+06,85000.0,375.0,1.067467e+04,0.021234
1236,DNK,Denmark,2021,False,4.983000e+06,1072000.0,12.0,4.152500e+05,0.215131
1237,FIN,Finland,2021,False,6.413000e+07,2003000.0,100.0,6.413000e+05,0.031233
1238,DEU,Germany,2021,False,1.064479e+09,223338000.0,9421.0,1.129900e+05,0.209810
1240,IRL,Ireland,2021,True,2.483220e+08,29860000.0,1473.0,1.685825e+05,0.120247
1241,ITA,Italy,2021,False,4.433000e+08,68797000.0,2231.0,1.987001e+05,0.155193
1242,LUX,Luxembourg,2021,True,2.935300e+07,11646000.0,312.0,9.408013e+04,0.396757
1244,NLD,Netherlands,2021,True,5.936200e+07,1011000.0,75.0,7.914933e+05,0.017031
1245,POL,Poland,2021,False,1.317500e+07,1860000.0,637.0,2.068289e+04,0.141176


## Transparency score

Purpose is to calculate a transparency score for each multinational and multinational-year (author Kane Borders).

In [21]:
# List financial columns 
financial_columns = [
    'total_revenues',
    'profit_before_tax',
    'tax_paid',
    'tax_accrued',
    'unrelated_revenues', 
    'related_revenues',
    'stated_capital',
    'accumulated_earnings',
    'tangible_assets',
    'employees'
]


def compute_geographic_score(df:pd.DataFrame, company:str, year:int) -> float:
    """Compute component I of transparency score which is the geographic score.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name.
        year (int): fiscal year to filter the results with.

    Returns:
        float: value of the score.
    """
    
    # Filter rows with selected company and subset with financial columns
    df = df.loc[
        (df['mnc'] == company) & (df['year'] == year),
        ['mnc', 'year', 'upe_code', 'jur_code', 'jur_name', *financial_columns]
    ]

    # Remove columns where data are missing for all jurisdictions
    df = df.dropna(axis='columns', how='all')

    # List financial columns left after deleting columns with only missing values
    financial_columns_left = [col for col in df.columns if col in financial_columns]

    # Geographic score = 0 if no financial columns left
    if not financial_columns_left:
        return 0
    
    # Get absolute values of financial data to have only "positive" values
    df[financial_columns_left] = df[financial_columns_left].abs()

    # Calculate percentage of each financial value where jurisdiction is 'OTHER'
    # Percentage = 1. Total of 'OTHER' row(s) / 2. Total of all rows
    other_percentage = (
        df.loc[df['jur_code'] == 'OTHER', financial_columns_left].sum() # 1
        / df[financial_columns_left].sum() # 2
    )

    # Calculate geographic score
    geographic_score = 100 - np.mean(other_percentage) * 100

    return geographic_score


def compute_completness_score(df:pd.DataFrame, company:str, year:int) -> float:
    """Compute component II of transparency score which is the completness score.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name.
        year (int): fiscal year to filter the results with.

    Returns:
        float: value of the score.
    """
    
    # Filter rows with selected company and subset with financial columns
    df = df.loc[
        (df['mnc'] == company) & (df['year'] == year),
        ['mnc', 'year', 'upe_code', 'jur_code', 'jur_name', *financial_columns]
    ]

    # Remove columns where data are missing for all jurisdictions
    df = df.dropna(axis='columns', how='all')

    # List financial columns left after deleting columns with only missing values
    financial_columns_left = [col for col in df.columns if col in financial_columns]

    # Completness score = 0 if no financial columns left
    if not financial_columns_left:
        return 0

    # Calculate score with weighting :
    # * 1 pts per financial columns ;
    # * extra 1 pts for 'profit_before_tax' column if present ;
    # * extra 1 pts for 'tax paid' column if present.

    score = len(financial_columns_left)

    for variable in ['profit_before_tax', 'tax_paid']:
        if variable in df.columns:
            score += 1

    # Calculate completness score
    completness_score = score / 12 * 100

    return completness_score


def compute_transparency_score(df:pd.DataFrame, company:str) -> dict:
    """Compute the transparency score which is the average of component I
    (geographic score) and component II (completness score).

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name.
        year (int): fiscal year to filter the results with.

    Returns:
        float: value of the score.
    """

    # List all years where the company as reported
    years_list = sorted(df.loc[df['mnc'] == company, 'year'].unique())

    # Initialize an empty dictionnary
    data = dict()

    # Calculate scores for each year and add them to the dictionnary
    for year in years_list:

        # Calculate components I and II
        geographic_score = compute_geographic_score(df=df, company=company, year=year)
        completness_score = compute_completness_score(df=df, company=company, year=year)

        # Calculate final score which is the average of components I and II
        transparency_score = (geographic_score + completness_score) / 2

        data[year] = {
            'mnc': company,
            'geographic_score': geographic_score,
            'completness_score': completness_score,
            'transparency_score': transparency_score
        }

    return data


def transparency_scores_to_csv(
        df: pd.DataFrame, csv_path:str='./') -> pd.DataFrame:
    """Compute transparency score for all companies and all years into a 
    DataFrame and export it to a csv file (optionnal).

    Args:
        df (pd.DataFrame): CbCRs database.
        csv_path (str, optional): Path of csv file. Defaults to './'.

    Returns:
        _type_: Scores for all companies and years.
    """
    
    # List all companies
    mnc_list = df['mnc'].unique()

    # Initialize an empty DataFrame
    mnc_df = pd.DataFrame()
    
    # Calculate tranparency scores for all companies and add them to the DataFrame
    for mnc in mnc_list:
        temp_df = pd.DataFrame.from_dict(
            compute_transparency_score(df=df, company=mnc), orient='index')
        
        mnc_df = pd.concat([mnc_df, temp_df])

    # Reset index and move 'mnc' columns in first position
    mnc_df = mnc_df.reset_index().rename(columns={'index': 'year'})
    mnc_df.insert(0, 'mnc', mnc_df.pop('mnc'))

    if csv_path:
        mnc_df.to_csv(csv_path + 'transparency_scores.csv', index=False)

    return mnc_df


def display_transparency_score(df:pd.DataFrame, company:str, year: int=None):
    """Display transparency score for specific company in a metric.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name.
        year (int): fiscal year to filter the results with.
    """
    
    # Compute data
    data = compute_transparency_score(df=df, company=company)

    # Create DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')

    # Reset index and move 'mnc' columns in first position
    df = df.reset_index().rename(columns={'index': 'year'})

    # When data are not filtered by year, the score is the average of all years
    score = (
        df.loc[df['year'] == year, 'transparency_score'].iloc[0] if year 
        else df['transparency_score'].mean()
    )

    # Create figure
    fig = go.Figure()

    # Add circular background
    fig.add_shape(
        type='circle',
        x0=0, y0=0, x1=1, y1=1,
        line_color='blue',
        fillcolor='blue',
        opacity=0.3
    )

    # Add indicator
    fig.add_trace(go.Indicator(
        mode='number',
        value=score,
        number={'suffix': '%', 'valueformat': '.0f', 'font': {'size': 54}},
        domain={'x': [0, 1], 'y': [0, 1]}
    ))

    # Update layout
    fig.update_layout(
        width=360,
        height=360)

    # Show figure
    fig.show()


def display_transparency_score_over_time(df:pd.DataFrame, company:str):
    """Display transparency scores over time for a specific company in a bar
    chart.

    Args:
        df (pd.DataFrame): CbCRs database.
        company (str): Company name.
    """

    # Compute data
    data = compute_transparency_score(df=df, company=company)

    # Create DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')

    # Reset index and move 'mnc' columns in first position
    df = df.reset_index().rename(columns={'index': 'year'})

    # Create figure
    fig = px.bar(df, x='year', y='transparency_score',
                    text_auto='.0f'
                   )

    # Update layout settings
    fig.update_layout(
        title='Transparency score over time',
        xaxis=dict(
            title=None,
            tickvals=df['year'].unique()
        ),
        yaxis=dict(
            title=None,
            showline=True,
            ticks='outside',
            linecolor='grey',
            tickcolor='grey',
            range=[0, 101],
            tickvals=[0, 25, 50, 75, 100],
            ticktext=[0, '', '', '', 100]
        ),
        plot_bgcolor='white',
        width=800,
        height=480        
    )

    # Force position and color of bar values
    fig.update_traces(
        textposition='outside', textfont=dict(color=fig.data[0].marker.color)
    )

    fig.show()

In [22]:
print(company)
display_transparency_score(df, company)

print(company, year)
display_transparency_score(df, company, year)

print(company)
display_transparency_score_over_time(df, company)

AXA


AXA 2018


AXA
